[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mckinsey/vizro/blob/docs/add_colab_link/vizro-ai/examples/dashboard_by_vizro_ai.ipynb)

# Use Vizro-AI to generate Vizro dashboard

In [ ]:
# @title Setup
#@markdown <-- click the `Run cell` button to
#@markdown #### 1. Install Vizro-AI
print("Installing vizro-ai")
#@markdown (TODO: change to !pip install vizro-ai when this PR is ready to merge)
# !pip install vizro-ai
!pip install "vizro-ai @ git+https://github.com/mckinsey/vizro.git@main#subdirectory=vizro-ai" -q -q -q
print("✅ vizro-ai installed")

#@markdown #### 2. Secure your LLM API key as environment variable
#@markdown - click the `Secrets` button on the left
#@markdown - click `Add new secret`
#@markdown - insert "OPENAI_API_KEY" under `Name`
#@markdown - insert the API key value under `Value`
#@markdown - [optional] you might also need to insert "OPENAI_BASE_URL" and its values, based on the requirements from your API key provider.
#@markdown - click the `Notebook access` toggle to make it available for the notebook run

import os
from google.colab import userdata

def safe_get_userdata(key):
    try:
        return userdata.get(key)
    except Exception as e:
        print(f"Warning: Unable to access {key}. Reason: {str(e)}")
        return None

# Always try to set the OPENAI_API_KEY
api_key = safe_get_userdata("OPENAI_API_KEY")
if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    print("❌ OPENAI_API_KEY not set. Click `Secrets` icon on the left to setup.")

# Conditionally set OPENAI_BASE_URL if provided and accessible
openai_base_url = safe_get_userdata("OPENAI_BASE_URL")
if openai_base_url:
    os.environ["OPENAI_BASE_URL"] = openai_base_url
else:
    print("User defined OPENAI_BASE_URL not set. Using default URL.")

print("\nCurrent environment variables:")
print(f"OPENAI_API_KEY: {'✅ Set' if 'OPENAI_API_KEY' in os.environ else 'Not set'}")
print(f"OPENAI_BASE_URL: {'✅ Set' if 'OPENAI_BASE_URL' in os.environ else 'Not set by user, use default'}")

In [ ]:
# @title ## Set user input

#@markdown <-- click the `Run cell` button to
#@markdown #### 1. Choose the LLM model

LLM = 'gpt-4o' # @param ["gpt-4o", "gpt-3.5-turbo", "gpt-4-turbo"]

# print(f"Selected LLM: {LLM}")

#@markdown ---
#@markdown #### 2. Upload your CSV data (multiple dataframes accepted)
#@markdown - click `Choose Files` if you'd like to upload and use your own data
#@markdown - or click `Cancel upload` and use the default example data for initial exploration

import pandas as pd
from plotly.express.data import gapminder
from google.colab import files

uploaded = files.upload()

dfs = []

if not uploaded:
    print("No files uploaded. Use the plotly gapminder dataset as an example.")
    df = gapminder()
    dfs.append(df)
else:
    for fn in uploaded.keys():
        print('User uploaded file "{name}"'.format(
            name=fn, length=len(uploaded[fn])))

        df_uploaded = pd.read_csv(fn)
        dfs.append(df_uploaded)

#@markdown ---
#@markdown #### 3. Input your chart question

# @markdown #### Instructions:
user_input = 'Create a gdp dashboard with 1 page. use a bar chart to show gdp per continent. add a filter to filter data by country' # @param {type:"string", placeholder:"Enter the requirements for the dashboard"}

# print(user_input)

In [ ]:
# @title 6. Build the dashboard using Vizro-AI

import ipywidgets as widgets
from IPython.display import display
# from vizro import Vizro

# Vizro._reset()
# fig = None

def run_code(button):
    from vizro_ai import VizroAI
    from vizro import Vizro

    Vizro._reset()
    vizro_ai = VizroAI(model=LLM)
    dashboard = vizro_ai.dashboard(dfs, user_input)

    vizro_dashboard = Vizro().build(dashboard)

    button1.result = vizro_dashboard

button1 = widgets.Button(description="Build Dashboard!")
display(button1)

button1.on_click(run_code)

In [ ]:
# @title 7. Render dashboard

import ipywidgets as widgets
from IPython.display import display

def run_dashboard(button):
    button1.result.run()

button2 = widgets.Button(description="Run Dashboard!")
display(button2)

button2.on_click(run_dashboard)